In [1]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate
import pandas as pd

In [2]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [4]:
# !./setuprds.sh
!./setupdynamo.sh

Creating the DynamoDB table that holds the client floats
{
    "StackId": "arn:aws:cloudformation:us-east-1:123456789:stack/client-float-table/67a65f1b-88ba-46f2-87a1-f6d957d40c7a"
}
Adding in item for core, ZAR wholesale float


In [5]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

Serverless Enterprise: Run `serverless login` and deploy again to explore, monitor, secure your serverless project for free.
Deploying the migration lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfuncti

In [6]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [7]:
# Setup the RDS roles
role_payload = { "type": "CREATE_ROLES", "credentials": {
        "account_api_worker": "pwd_for_account_api",
        "save_tx_api_worker": "pwd_for_transaction_api",
        "float_api_worker": "pwd_for_float_api",
        "auth_api_worker": "pwd_for_auth_api_worker"
    }
}
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps(role_payload))

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '372',
   'access-control-allow-origin': '*',
   'server': 'Werkzeug/0.15.4 Python/3.6.8',
   'date': 'Mon, 01 Jul 2019 20:59:44 GMT'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'Payload': <botocore.response.StreamingBody at 0x7f6875910198>}

In [8]:
# Setup the RDS tables
local_lambda.invoke(FunctionName='db-migration-local-migrate', InvocationType='RequestResponse', Payload=json.dumps({'type': 'SETUP_TABLES'}))

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '86',
   'access-control-allow-origin': '*',
   'server': 'Werkzeug/0.15.4 Python/3.6.8',
   'date': 'Mon, 01 Jul 2019 20:59:57 GMT'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'Payload': <botocore.response.StreamingBody at 0x7f6875910208>}

In [9]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [10]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'floatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [11]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [12]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='account-create', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [13]:
def generate_saving_transaction(account_id, ref_amount = 100, client_id = 'zar_client_co', float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id,
        'clientId': client_id
    }
    return test_saving_dict

In [14]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='activity-save', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Tx for account %s persisted with new balance %s' % (tx['accountId'], persisted_tx['currentBalance']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [15]:
def run_accrual_for_float(client_id = 'zar_client_co', float_id = 'zar_cash_primary', accrued_amount = 100):
    accrualDict = {
        'clientId': client_id,
        'floatId': float_id,
        'accrualAmount': accrued_amount,
        'currency': 'ZAR',
        'unit': 'HUNDREDTH_CENT',
        'backingEntityIdentifier': 'tx-id-backing'
    }
    lambda_result = local_lambda.invoke(FunctionName='float-api-local-accrue', InvocationType='RequestResponse', 
                                               Payload=json.dumps(accrualDict, default=str))
    lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
    return lambda_payload

In [16]:
print_deployed_functions()

Function list:  ['float-accrue', 'account-create', 'activity-save', 'activity-balance', 'db-migration-local-migrate']


In [17]:
accounts = create_number_accounts(number_accounts = 10)

Account with ID 5c5d192c-7d76-472d-b115-a364d21b96e0 persisted at 2019-07-01T21:01:05.335Z
Account with ID ca49d6ec-5a65-41f3-a2d8-78fb3f0321ab persisted at 2019-07-01T21:01:06.337Z
Account with ID c8021d33-5d2a-4b01-9ecb-a818d0973bb6 persisted at 2019-07-01T21:01:07.177Z
Account with ID 4ab408d3-215e-4449-8688-62fa14ca11fb persisted at 2019-07-01T21:01:08.084Z
Account with ID 6939f226-db37-4710-a4dd-640ddd5f819f persisted at 2019-07-01T21:01:09.014Z
Account with ID 03dce586-32da-4279-b8f4-d24044eff052 persisted at 2019-07-01T21:01:09.857Z
Account with ID 795ed8aa-3f5f-41a8-abbc-d2fb11339f85 persisted at 2019-07-01T21:01:10.690Z
Account with ID 270a9abf-1607-486a-92a0-fbc5007b3bfc persisted at 2019-07-01T21:01:11.604Z
Account with ID ee373fcd-c866-4e1c-bae9-f34cc39d39d3 persisted at 2019-07-01T21:01:12.616Z
Account with ID 1ddccc2e-7fbc-4ab2-bb39-1046f6da7f15 persisted at 2019-07-01T21:01:13.478Z


In [18]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts[:100]]
print(tabulate.tabulate(account_info, headers = 'keys'))

Name                AccountId
------------------  ------------------------------------
Jacob Gross         5c5d192c-7d76-472d-b115-a364d21b96e0
Jennifer Weatherby  ca49d6ec-5a65-41f3-a2d8-78fb3f0321ab
Bethel Duck         c8021d33-5d2a-4b01-9ecb-a818d0973bb6
Earl Chase          4ab408d3-215e-4449-8688-62fa14ca11fb
Leonard Roth        6939f226-db37-4710-a4dd-640ddd5f819f
Roy Walker          03dce586-32da-4279-b8f4-d24044eff052
Michelle Mueller    795ed8aa-3f5f-41a8-abbc-d2fb11339f85
Terry Hinkley       270a9abf-1607-486a-92a0-fbc5007b3bfc
Glennie Brooks      ee373fcd-c866-4e1c-bae9-f34cc39d39d3
Roman Soares        1ddccc2e-7fbc-4ab2-bb39-1046f6da7f15


In [34]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='activity-save', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

Lambda result {'body': '{"transactionDetails":[[{"transaction_id":"16cd33f0-a80d-4e43-a50a-96a5448e4518","creation_time":"2019-07-01T21:55:42.646Z"}],[{"transaction_id":"c673e3bc-78da-4d44-90d8-f450090e4bd0","creation_time":"2019-07-01T21:55:42.646Z"},{"transaction_id":"c673e3bc-78da-4d44-90d8-f450090e4bd0","creation_time":"2019-07-01T21:55:42.646Z"}]],"newBalance":66963}', 'statusCode': 200}


In [47]:
balance_payload = { 'accountId': accounts[0]['accountId'], 'clientId': 'zar_client_co', 'floatId': 'zar_cash_primary' }
balance_projection = local_lambda.invoke(FunctionName='activity-balance', Payload=json.dumps(balance_payload))
projection_decoded = decode_lambda_result(balance_projection)
print(projection_decoded)

{'balanceEndOfToday': {}, 'currentBalance': {}}


In [ ]:
account_balances = seed_savings_for_accounts(accounts[:100], tx_per_account = 3, base_amount = 100 * 100 * 100)

In [ ]:
def merge_dataframes(accounts, account_balances):
    df = pd.DataFrame(account_balances)
    df = df[df['currentBalance'] == df.groupby('accountId')['currentBalance'].transform('max')]
    df = df[['accountId', 'currentBalance']]
    df['currentBalance'] = pd.to_numeric(df['currentBalance'])
    df['balanceRand'] = df['currentBalance']/10000
    account_name_df = pd.DataFrame(accounts)[['accountId', 'floatId', 'userFamilyName', 'userFirstName']]
    account_name_df['Holder Name'] = account_name_df['userFirstName'] + ' ' + account_name_df['userFamilyName']
    account_name_df.set_index('accountId')
    df.set_index('accountId')
    merged_df = pd.merge(account_name_df, df)[['accountId', 'floatId', 'Holder Name', 'balanceRand']]
    return merged_df

In [ ]:
merged_df = merge_dataframes(accounts, account_balances)

In [ ]:
merged_df

In [ ]:
amount_to_accrue = round(random.random() * 100 * 100 * 100)
print('Instructing accrual of : R', amount_to_accrue / (100 * 100))
accrual_result = run_accrual_for_float(accrued_amount = amount_to_accrue)
allocation_details = json.loads(accrual_result['userAllocationTransactions']['body'])
account_transaction_list = allocation_details['allocationRecords']['accountTxIds']
# print(tabulate.tabulate(account_transaction_list, headers = 'keys'))

In [ ]:
# print(accrual_result)
account_transactions = allocation_details['allocationRecords']['accountTxIds']
number_of_float_transactions = len(allocation_details['allocationRecords']['floatTxIds'])
number_of_account_transactions = len(account_transactions)
print('%d float transactions, %d account transactions' % (number_of_float_transactions, number_of_account_transactions))
sum_of_accounts = sum([transaction['amount'] for transaction in account_transactions])
print('Sum of account additions: ', sum_of_accounts)
bonus_allocation = accrual_result['entityAllocations']['bonusShare']
company_allocation = accrual_result['entityAllocations']['clientShare']
print('Bonus allocation: %d, and client co allocation: %d' % (bonus_allocation, company_allocation))
total_allocations = sum_of_accounts + bonus_allocation + company_allocation
print('Of accrual %d, allocations %d' % (amount_to_accrue, total_allocations))
rounding_allocated_to_bonus = isinstance(allocation_details['bonusAllocation'], list)
print('Was there a rounding allocation? : ', rounding_allocated_to_bonus)
print('Rounding allocation record: ', allocation_details['bonusAllocation'])